# Import

In [41]:
from vizdoom import *
import random
import time
import numpy as np
import gym
from gym import Env
from gym.spaces import Discrete, Box
import cv2
# Import callback class from sb3
from stable_baselines3.common.callbacks import BaseCallback
import os
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common import policies
# import ppo for training
from stable_baselines3 import PPO
from stable_baselines3 import DQN

# Testing Defend the Center game

In [42]:
game = DoomGame()
game.load_config('VizDoom/scenarios/defend_the_center.cfg')
game.init()

In [43]:
# This is the set of actions we can take in the environment
actions = np.identity(3, dtype=np.uint8)

In [44]:
state = game.get_state()
state.game_variables

array([ 26., 100.])

In [45]:
episodes = 10 
for episode in range(episodes): 
    # Create a new episode or game 
    game.new_episode()
    # Check the game isn't done 
    while not game.is_episode_finished(): 
        # Get the game state 
        state = game.get_state()
        # Get the game image 
        img = state.screen_buffer
        # Get the game variables - ammo
        info = state.game_variables
        # Take an action
        reward = game.make_action(random.choice(actions),4)
        # Print rewward 
        print('reward:', reward) 
        time.sleep(0.02)
    print('Result:', game.get_total_reward())
    time.sleep(2)


reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 1.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 1.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: -1.0
Result: 1.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
rew

reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 1.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: -1.0
Result: 2.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
reward: 0.0
rew

In [46]:
game.close()

# Set-up openAI framework

In [7]:
import torch

In [8]:
if torch.cuda.is_available():
    device = torch.device("cuda")          # a CUDA device object
    print('CUDA is available on this device!')
else:
    device = torch.device("cpu")           # a CPU device object
    print('CUDA is not available on this device :(')

CUDA is available on this device!


In [47]:
# Create DOOM OpenAI Gym SIMPLE Environment
class VizDoomGym(Env): 
    def __init__(self, render=False): 
        super().__init__()
        self.game = DoomGame()
         
        #load basic configuration for simple environment
        self.game.load_config('VizDoom/scenarios/defend_the_center.cfg')
        
        #Set visibility of game
        if render == True: 
            self.game.set_window_visible(True)
        else:
            self.game.set_window_visible(False)
        
        # Start the game 
        self.game.init()
        
        # Create observation space
        self.observation_space = Box(low=0, high=255, shape=(100,160,1), dtype=np.uint8) 
        
        # Create action space
        self.action_space = Discrete(3)
        
    # Create Step function
    def step(self, action):
        actions = np.identity(3)
        reward = self.game.make_action(actions[action], 4) 
        
        # Get information from the game
        if self.game.get_state(): 
            state = self.game.get_state().screen_buffer
            state = self.grayscale(state)
            ammo = self.game.get_state().game_variables[0]
            info = ammo
        else: 
            state = np.zeros(self.observation_space.shape)
            info = 0 
        
        info = {"info":info}
        done = self.game.is_episode_finished()
        
        return state, reward, done, info 
    
    # Render
    def render(): 
        pass
    
    # Reset game
    def reset(self): 
        self.game.new_episode()
        state = self.game.get_state().screen_buffer
        return self.grayscale(state)
    
    # Grayscale the game frame and resize it 
    def grayscale(self, observation):
        gray = cv2.cvtColor(np.moveaxis(observation, 0, -1), cv2.COLOR_BGR2GRAY)
        resize = cv2.resize(gray, (160,100), interpolation=cv2.INTER_CUBIC)
        state = np.reshape(resize, (100,160,1))
        return state
    
    # Call to close down the game
    def close(self): 
        self.game.close()

In [7]:
env = VizDoomGym(render=True)

In [11]:
state = env.reset()

In [9]:
env.close()

# Create Callback

In [16]:
class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'model_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True

# Hyper parameter Tuning

In [10]:
import optuna

In [11]:
def optimise_ppo(trial):
    """ Learning hyperparamters we want to optimise"""
    return {
        'n_steps': int(trial.suggest_loguniform('n_steps', 32, 8192)),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 0.1),
        'ent_coef': trial.suggest_loguniform('ent_coef', 1e-7, 1e-1),
    }

def optimise_agent(trial):
    model_params = optimise_ppo(trial)
    env = VizDoomGym(render=False)
    model = PPO('CnnPolicy', env, verbose =1, **model_params)
    model.learn(30000)
    
    
    rewards = []
    n_episodes, reward_sum = 0, 0.0

    obs = env.reset()
    while n_episodes < 4:
        action, _ = model.predict(obs)
        obs, reward, done, info = env.step(action)
        reward_sum += reward

        if done:
            rewards.append(reward_sum)
            reward_sum = 0.0
            n_episodes += 1
            obs = env.reset()

    last_reward = np.mean(rewards)

    return -1 * last_reward
    
    

In [12]:
study = optuna.create_study()

[I 2023-04-09 15:27:02,120] A new study created in memory with name: no-name-a82f948a-d341-4fee-a368-827c43be7015


In [13]:
study.optimize(optimise_agent, n_trials = 15)

C:\Users\Ga401\AppData\Local\Temp\ipykernel_33148\1157592159.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'n_steps': int(trial.suggest_loguniform('n_steps', 32, 8192)),
C:\Users\Ga401\AppData\Local\Temp\ipykernel_33148\1157592159.py:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 0.1),
C:\Users\Ga401\AppData\Local\Temp\ipykernel_33148\1157592159.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'ent_

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.


C:\Users\Ga401\anaconda3\lib\site-packages\stable_baselines3\ppo\ppo.py:145: UserWarning: You have specified a mini-batch size of 64, but because the `RolloutBuffer` is of size `n_steps * n_envs = 184`, after every 2 untruncated mini-batches, there will be a truncated mini-batch of size 56
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=184 and n_envs=1)
  warnings.warn(


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 82       |
|    ep_rew_mean     | 0        |
| time/              |          |
|    fps             | 22       |
|    iterations      | 1        |
|    time_elapsed    | 8        |
|    total_timesteps | 184      |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 76.2         |
|    ep_rew_mean          | -0.25        |
| time/                   |              |
|    fps                  | 26           |
|    iterations           | 2            |
|    time_elapsed         | 13           |
|    total_timesteps      | 368          |
| train/                  |              |
|    approx_kl            | 2.815273e-05 |
|    clip_fraction        | 0            |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.1         |
|    explained_variance   | -0.00272     |
|    learning_r

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 76.7         |
|    ep_rew_mean          | -0.115       |
| time/                   |              |
|    fps                  | 32           |
|    iterations           | 11           |
|    time_elapsed         | 62           |
|    total_timesteps      | 2024         |
| train/                  |              |
|    approx_kl            | 8.188809e-06 |
|    clip_fraction        | 0            |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.1         |
|    explained_variance   | 0.505        |
|    learning_rate        | 2.66e-05     |
|    loss                 | 0.0642       |
|    n_updates            | 100          |
|    policy_gradient_loss | -4.43e-05    |
|    value_loss           | 0.139        |
------------------------------------------
-------------------------------------------
| rollout/                |               |
|    ep_l

...

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 109         |
|    ep_rew_mean          | 2.71        |
| time/                   |             |
|    fps                  | 32          |
|    iterations           | 191         |
|    time_elapsed         | 802         |
|    total_timesteps      | 25976       |
| train/                  |             |
|    approx_kl            | 0.016347047 |
|    clip_fraction        | 0.134       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.692      |
|    explained_variance   | 0.642       |
|    learning_rate        | 3.2e-05     |
|    loss                 | -0.039      |
|    n_updates            | 1900        |
|    policy_gradient_loss | -0.0261     |
|    value_loss           | 0.258       |
-----------------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 109 

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 109         |
|    ep_rew_mean          | 2.67        |
| time/                   |             |
|    fps                  | 32          |
|    iterations           | 201         |
|    time_elapsed         | 846         |
|    total_timesteps      | 27336       |
| train/                  |             |
|    approx_kl            | 0.027863605 |
|    clip_fraction        | 0.285       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.575      |
|    explained_variance   | 0.12        |
|    learning_rate        | 3.2e-05     |
|    loss                 | 0.0576      |
|    n_updates            | 2000        |
|    policy_gradient_loss | -0.0206     |
|    value_loss           | 0.195       |
-----------------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 109 

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 109          |
|    ep_rew_mean          | 2.69         |
| time/                   |              |
|    fps                  | 32           |
|    iterations           | 211          |
|    time_elapsed         | 886          |
|    total_timesteps      | 28696        |
| train/                  |              |
|    approx_kl            | 0.0042033764 |
|    clip_fraction        | 0.0161       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.561       |
|    explained_variance   | 0.847        |
|    learning_rate        | 3.2e-05      |
|    loss                 | -0.0684      |
|    n_updates            | 2100         |
|    policy_gradient_loss | -0.00981     |
|    value_loss           | 0.125        |
------------------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 109          |
|    ep_rew_mean          | 2.76         |
| time/                   |              |
|    fps                  | 32           |
|    iterations           | 221          |
|    time_elapsed         | 929          |
|    total_timesteps      | 30056        |
| train/                  |              |
|    approx_kl            | 0.0043340162 |
|    clip_fraction        | 0.0474       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.545       |
|    explained_variance   | 0.638        |
|    learning_rate        | 3.2e-05      |
|    loss                 | 0.109        |
|    n_updates            | 2200         |
|    policy_gradient_loss | -0.00618     |
|    value_loss           | 0.323        |
------------------------------------------


[I 2023-04-09 19:23:29,206] Trial 14 finished with value: -2.5 and parameters: {'n_steps': 136.91301589705148, 'learning_rate': 3.196228146849964e-05, 'ent_coef': 2.3022189875406435e-06}. Best is trial 12 with value: -10.75.


In [14]:
print(study.best_params)

{'n_steps': 1553.7406595657749, 'learning_rate': 0.00021026745671182018, 'ent_coef': 3.3643903698286066e-06}


# Training Agent

-PPO sensitive to learning rate and batch size  
-DQN sensitive to learning rate, Exploration rate, Batch size, Replay Buffer,Discount Factor

In [49]:
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
from stable_baselines3.common.env_util import make_vec_env

In [58]:
env = VizDoomGym(False)

In [59]:
CHECKPOINT_DIR = './model/center_model/PPO_model_center/'
LOG_DIR = './model_log/log_center'

callback = TrainAndLoggingCallback(check_freq=10000, save_path=CHECKPOINT_DIR)

In [60]:
model = PPO('CnnPolicy', env, tensorboard_log=LOG_DIR, verbose=1,
            learning_rate=0.0001, n_steps=4096, batch_size = 64, 
            ent_coef=0.00001)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.


In [61]:
model.learn(total_timesteps=150000, callback=callback)

Logging to ./model_log/log_center\PPO_3
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 77.5     |
|    ep_rew_mean     | 0.192    |
| time/              |          |
|    fps             | 34       |
|    iterations      | 1        |
|    time_elapsed    | 118      |
|    total_timesteps | 4096     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 84.1        |
|    ep_rew_mean          | 0.515       |
| time/                   |             |
|    fps                  | 31          |
|    iterations           | 2           |
|    time_elapsed         | 256         |
|    total_timesteps      | 8192        |
| train/                  |             |
|    approx_kl            | 0.010756422 |
|    clip_fraction        | 0.093       |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.09       |
|    explained_variance   | 0.00

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 143         |
|    ep_rew_mean          | 6.28        |
| time/                   |             |
|    fps                  | 31          |
|    iterations           | 11          |
|    time_elapsed         | 1449        |
|    total_timesteps      | 45056       |
| train/                  |             |
|    approx_kl            | 0.024996638 |
|    clip_fraction        | 0.237       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.789      |
|    explained_variance   | 0.856       |
|    learning_rate        | 0.0001      |
|    loss                 | 0.003       |
|    n_updates            | 100         |
|    policy_gradient_loss | -0.0421     |
|    value_loss           | 0.155       |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 144   

---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 198       |
|    ep_rew_mean          | 10.6      |
| time/                   |           |
|    fps                  | 30        |
|    iterations           | 21        |
|    time_elapsed         | 2809      |
|    total_timesteps      | 86016     |
| train/                  |           |
|    approx_kl            | 0.0309354 |
|    clip_fraction        | 0.226     |
|    clip_range           | 0.2       |
|    entropy_loss         | -0.604    |
|    explained_variance   | 0.951     |
|    learning_rate        | 0.0001    |
|    loss                 | -0.0284   |
|    n_updates            | 200       |
|    policy_gradient_loss | -0.0401   |
|    value_loss           | 0.115     |
---------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 201         |
|    ep_rew_mean          | 10.9  

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 233         |
|    ep_rew_mean          | 13.8        |
| time/                   |             |
|    fps                  | 28          |
|    iterations           | 31          |
|    time_elapsed         | 4416        |
|    total_timesteps      | 126976      |
| train/                  |             |
|    approx_kl            | 0.034676485 |
|    clip_fraction        | 0.217       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.479      |
|    explained_variance   | 0.952       |
|    learning_rate        | 0.0001      |
|    loss                 | -0.0266     |
|    n_updates            | 300         |
|    policy_gradient_loss | -0.0344     |
|    value_loss           | 0.104       |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 230   

# Evaluate

In [62]:
# Import eval policy to test agent
from stable_baselines3.common.evaluation import evaluate_policy

In [64]:
# Reload model from disc
model = PPO.load('./model/center_model/PPO_model_center/model_150000')

In [33]:
# Create rendered environment
env = VizDoomGym(render=False)

In [82]:
# Evaluate mean reward for 10 games
mean_reward, _ = evaluate_policy(model, env, n_eval_episodes=100)

C:\Users\Ga401\anaconda3\lib\site-packages\stable_baselines3\common\evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


In [83]:
mean_reward

84.57

In [65]:
# Create rendered environment
env = VizDoomGym(render=True)

In [66]:
for episode in range(10): 
    obs = env.reset()
    done = False
    result_reward = 0
    while not done: 
        action, _ = model.predict(obs)
        obs, reward, done, info = env.step(action)
        # time.sleep(0.20)
        result_reward += reward
    print('Episode {}: Total Reward is {}'.format(episode, result_reward))
    time.sleep(1)

Total Reward for episode 15.0 is 0
Total Reward for episode 16.0 is 1
Total Reward for episode 13.0 is 2
Total Reward for episode 15.0 is 3
Total Reward for episode 19.0 is 4
Total Reward for episode 16.0 is 5
Total Reward for episode 13.0 is 6
Total Reward for episode 10.0 is 7
Total Reward for episode 17.0 is 8
Total Reward for episode 12.0 is 9
